In [1]:
import pandas as pd
import plotly.graph_objects as go
import wbdata

df1 = pd.read_csv('new_water_done5.csv')

df2 = pd.read_csv('iso_mei.csv')
industry_data = df2.rename(columns={'Time': 'Year'})

countries_gdp = wbdata.get_dataframe(indicators={'NY.GDP.MKTP.CD': 'GDP'}, convert_date=False)

countries_industry = industry_data['Country'].unique()

common_countries = list(set(countries_gdp.index.get_level_values('country').unique()) & set(countries_industry))

gdp_data = countries_gdp.loc[common_countries]
industry_data = industry_data[industry_data['Country'].isin(common_countries)]

gdp_data = gdp_data.pivot_table(index='date', columns='country', values='GDP')

num_bins = 3
bin_labels = ['Low GDP', 'Medium GDP', 'High GDP']

gdp_bins = pd.qcut(gdp_data.loc['2000'], num_bins, labels=bin_labels)

gdp_bin_countries = [list(gdp_bins[gdp_bins == label].index) for label in bin_labels]

avg_industry_data = industry_data.groupby(['Year', 'Country'])['Production of total industry sa, Index'].mean().unstack()

avg_industry_values = []
for bin_countries in gdp_bin_countries:
    avg_industry_bin = avg_industry_data[bin_countries].mean(axis=1)
    avg_industry_values.append(avg_industry_bin)

figgdp = go.Figure()

for bin_label, avg_values in zip(bin_labels, avg_industry_values):
    figgdp.add_trace(go.Scatter(x=avg_values.index, y=avg_values, name=bin_label))

figgdp.update_layout(
    xaxis_title='Year',
    yaxis_title='Average Production of total industry sa, Index',
    title='Average Industry Production by GDP Bin',
    legend_title='GDP Bin'
)

%store figgdp

figgdp.show()













ModuleNotFoundError: No module named 'wbdata'